Design Effectiveness
- Was the experiment well-structured?
- Were clients randomly and equally divided between the old and new designs?
- Were there any biases?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
df_merge = pd.read_csv('../../data/clean/final_clean_client_df.csv')
# df_merge

In [ ]:
# Estadísticas descriptivas por grupo
df_merge.groupby('variation').agg({
    'balance': ['mean', 'std'],
    'tenure_years': ['mean', 'std'],
    'age': ['mean', 'std'],
    'accounts': ['mean', 'std'],
    'calls_last_6_months': ['mean', 'std'],
    'logons_last_6_months': ['mean', 'std']
})

In [ ]:
# display_names = {
#     'balance': 'Account Balance',
#     'tenure_years': 'Years of Tenure',
#     'age': 'Customer Age',
#     'accounts': 'Number of Accounts'
# }

# variables = ['balance', 'tenure_years', 'age', 'accounts']
# for var in variables:
#     sns.histplot(data=df_merge, x=var, hue='variation', kde=True)
#     plt.title(f'Distribution of {display_names[var]} by Variation')
#     plt.xlabel(display_names[var])
#     plt.show()

In [ ]:
# Comparar balance entre los grupos
anova_balance = f_oneway(
    df_merge[df_merge['variation'] == 'Control']['balance'],
    df_merge[df_merge['variation'] == 'Test']['balance'],
    df_merge[df_merge['variation'] == 'Unknown']['balance']
)
print('ANOVA for balance:', anova_balance)

Pregunta: Como podria saber cuales fueron estas pequeñas diferencias que sugiere ANOVA?

Para identificar las pequeñas diferencias específicas que sugiere el ANOVA, puedes realizar una prueba post hoc. Una de las más utilizadas es la prueba de Tukey HSD (Honestly Significant Difference), que identifica cuáles pares de grupos tienen diferencias estadísticamente significativas.

In [ ]:
# Preparar los datos
balance_values = df_merge['balance']  # Valores del balance
variation_groups = df_merge['variation']  # Grupos de variación

# Realizar la prueba de Tukey
tukey_result = pairwise_tukeyhsd(endog=balance_values, groups=variation_groups, alpha=0.05)

# Mostrar los resultados
print(tukey_result)

In [ ]:
# tukey_result.plot_simultaneous(comparison_name='Control', figsize=(8, 6))
# plt.title('Tukey HSD for Balance by Variation')
# plt.show()

Interpretación gráficos

Distribución del balance:
- Todas las variaciones muestran distribuciones altamente sesgadas hacia balances bajos. Esto implica que la mayoría de los clientes tienen balances más bajos, independientemente del grupo.
- Visualmente, no hay diferencias evidentes entre los grupos, lo que sugiere que el ANOVA detectó una diferencia significativa en los promedios que no es fácil de observar directamente.

Distribución de Tenure Years:
- La distribución es similar para todos los grupos, con un pico en los clientes con menos de 10 años de antigüedad y una disminución gradual a medida que aumenta el número de años.

Distribución de la edad:
- Aunque los grupos tienen distribuciones similares, el grupo Unknown parece tener una mayor proporción de clientes jóvenes (20-30 años).

Distribución del número de cuentas:
- La mayoría de los clientes tienen 2 cuentas. Las distribuciones son muy similares entre los grupos.

Conclusión

El experimento parece estar bien equilibrado en términos de métricas clave como antigüedad, edad y número de cuentas.
Aunque el ANOVA muestra diferencias significativas en el balance promedio, la magnitud de la diferencia parece ser pequeña y no necesariamente significativa en términos prácticos. Esto sugiere que el diseño del experimento es razonablemente equitativo, aunque podría haber pequeñas diferencias que deberían ser consideradas en los análisis.

Conclusión 2

Con base en los resultados obtenidos, tanto del ANOVA como del test de comparación de Tukey:

Diferencias significativas entre grupos:

El ANOVA indicó que existen diferencias estadísticamente significativas en los balances promedio entre los grupos (p-value ≈ 2.67e-11).
El test de Tukey confirma cuáles son las comparaciones entre grupos que presentan diferencias significativas:

- Control vs Test: Existe una diferencia positiva significativa en el balance promedio (meandiff ≈ 3432.54), lo que indica que el grupo de prueba (Test) tiene balances promedio mayores que el grupo control.
- Control vs Unknown: Existe una diferencia negativa significativa (meandiff ≈ -5195.64), lo que indica que el grupo Unknown tiene balances promedio menores que el grupo control.
- Test vs Unknown: También se observa una diferencia negativa significativa (meandiff ≈ -8628.18), lo que significa que el grupo Unknown tiene balances promedio menores que el grupo de prueba.

Visualización del gráfico de Tukey:

El gráfico refleja que el balance promedio del grupo Test es mayor que el de Control, mientras que el grupo Unknown tiene el promedio más bajo de los tres.
Las barras de error no se solapan significativamente, lo que respalda las diferencias detectadas por los tests estadísticos.

Interpretación:

El análisis sugiere que las diferencias en el balance promedio entre los grupos son significativas, lo cual podría impactar la equidad del experimento. Es decir, los grupos no parecen estar completamente equilibrados respecto a esta variable, especialmente considerando el menor balance promedio del grupo Unknown.

Implicaciones:

Diseño del experimento:
- Las diferencias en los balances iniciales podrían influir en los resultados del experimento, ya que los usuarios con diferentes niveles de balance podrían comportarse de manera distinta.
- Sería recomendable ajustar los análisis posteriores considerando esta disparidad o controlar estas variables para interpretar correctamente el impacto del nuevo diseño.

Acciones futuras:
- Analizar si otras variables como la antigüedad del cliente (tenure_years) o el número de cuentas (num_accounts) muestran un patrón similar de desequilibrio entre los grupos.
- Considerar un rediseño del experimento para asegurar que los grupos sean más homogéneos o utilizar técnicas estadísticas como el análisis de covarianza (ANCOVA) para ajustar por las diferencias iniciales.

En resumen, aunque el experimento muestra diferencias significativas entre los grupos, estas diferencias iniciales podrían sesgar los resultados y deben tomarse en cuenta en cualquier interpretación o recomendación basada en el experimento.